In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from utility import calculate_input_gradients, perturb_inputs, preprocess_images, \
                    postprocess_features, save_data_hdf5,get_dataset_hdf5,\
                    build_one_class_svm, combine_inliners_outliers, apply_temp_scale_to_model,\
                    apply_log_temp_scale_to_model, perturb_inputs_odin, extract_layer_features
 
from utility_db_outliers import load_dataset
from models_lib import load_custom_model_for_ds
import h5py
# from metrics2 import *
from metrics import *
from general_setting import *
import time
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
#------------
from utility_methods import *
# from utility_methods2 import *
from sklearn.metrics import roc_auc_score

from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import scipy.io as sio
import tensorflow.keras.backend as K
import math

/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3

In [10]:
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten, Lambda
from tensorflow.keras.layers import Reshape, Conv2DTranspose, MaxPooling2D, UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [2]:
BATCH_SIZE = 100
SAVE_RESULTS = True
id_name=ID_DS_LIST[1] # selects the ID dataset.
id_model=ID_MODEL_LIST[1]  # select the deep model used for training ID dataset.
print(id_name,id_model)

CIFAR10 VGG


In [3]:
(org_traing_data, org_training_labels),(id_eva_data,org_testing_labels)  = load_dataset(id_name)

org_traing_data_processed = preprocess_images(id_name, org_traing_data, id_model, verbose=True)
id_eva_data_processed = preprocess_images(id_name, id_eva_data, id_model, verbose=True)

Preprocessing was done for  CIFAR10 VGG
Preprocessing was done for  CIFAR10 VGG


In [4]:
x_train = org_traing_data_processed
y_train = org_training_labels
x_test = id_eva_data_processed
y_test = org_testing_labels

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler( )
scaler.fit(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test_0 = scaler.transform(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test = x_test_0.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],x_test.shape[3])

In [6]:
scaler.fit(x_train.flatten().reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]*x_train.shape[3]))
x_train_0 = scaler.transform(x_train.flatten().reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]*x_train.shape[3]))
x_train = x_train_0.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],x_train.shape[3])

In [7]:
# 网络参数
input_shape = (32, 32, 3)
batch_size = 256
kernel_size = 3
# filters = 16  
latent_dim = 1024 # 隐变量取2维只是为了方便后面画图
epochs = 100
x_in = Input(shape=input_shape)
x = x_in

In [8]:
structure = [32, 64, 128]

In [11]:
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                activation='relu',
                strides=2,
                padding='same')(x)
    # elif filters == "max":
    #     x = MaxPooling2D((2, 2), padding="same")(x)
    # elif filters == "average":
    #     x = AveragePooling2D((2, 2), padding="same")(x)

# 备份当前shape，等下构建decoder的时候要用
shape = K.int_shape(x)

x = Flatten()(x)
x = Dense(4*4*128, activation='relu')(x)
# 算p(Z|X)的均值和方差
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# 重参数技巧
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=K.shape(z_mean))
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [12]:
# 重参数层，相当于给输入加入噪声
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# 解码层，也就是生成器部分
# 先搭建为一个独立的模型，然后再调用模型
latent_inputs = Input(shape=(latent_dim,))
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

In [13]:
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2DTranspose(filters=filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=2,
                    padding='same')(x)
    # elif filters == "max" or filters == "average":
    #             x = UpSampling2D((2, 2))(x)

outputs = Conv2DTranspose(filters=3,
                        kernel_size=kernel_size,
                        activation='sigmoid',
                        padding='same')(x)

In [14]:
# 搭建为一个独立的模型
encoder = Model(x_in, z_mean)

decoder = Model(latent_inputs, outputs)

x_out = decoder(z)

In [15]:
# 建立模型
vae = Model(x_in, x_out)

In [16]:
# xent_loss是重构loss，kl_loss是KL loss
xent_loss = K.sum(K.binary_crossentropy(x_in, x_out), axis=[1, 2, 3])
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)

# add_loss是新增的方法，用于更灵活地添加各种loss
vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 8, 8, 64)     18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 4, 4, 128)    73856       conv2d_1[0][0]                   
__________________________________________________________________________________________________
flatten (F

In [17]:
vae.fit(x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 8s 169us/step - loss: 2048.8856 - val_loss: 1968.1406
Epoch 2/100
50000/50000 [==============================] - 6s 118us/step - loss: 1968.0286 - val_loss: 1952.7644
Epoch 3/100
50000/50000 [==============================] - 6s 118us/step - loss: 1948.6853 - val_loss: 1951.3068
Epoch 4/100
50000/50000 [==============================] - 6s 117us/step - loss: 1935.3231 - val_loss: 1958.6573
Epoch 5/100
50000/50000 [==============================] - 6s 117us/step - loss: 1929.2501 - val_loss: 1917.8004
Epoch 6/100
50000/50000 [==============================] - 6s 117us/step - loss: 1923.1278 - val_loss: 1928.8299
Epoch 7/100
50000/50000 [==============================] - 6s 117us/step - loss: 1915.2705 - val_loss: 1924.8960
Epoch 8/100
50000/50000 [==============================] - 6s 117us/step - loss: 1908.4289 - val_loss: 1923.2063
Epoch 9/100
50000/50000 [=====================

In [18]:
vae.save('./vae_models/%s_%s.h5'%(id_name,id_model))
vae.save_weights('./vae_models/%s_%s_weights.h5'%(id_name,id_model))

In [20]:
def mark(x_1, x_2):
    diff = np.abs(x_1 - x_2)
    marks = np.mean(np.power(diff, 2), axis=(1,2,3))
    return marks

In [26]:
x_test = np.load('./advs_new/%s_%s_normal.npy'%(id_name, id_model),allow_pickle=True)
scaler.fit(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test_0 = scaler.transform(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test = x_test_0.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],x_test.shape[3])
print(x_test.shape)

(9347, 32, 32, 3)


In [27]:
sc_id = []
for i in range(len(x_test)):
    x_test_encoded = encoder.predict(x_test[i].reshape(1,32,32,3))
    xx = decoder.predict(x_test_encoded)
    score = mark(x_test[i],xx)
    # print(score)
    sc_id.append(-score)

In [22]:
def combine_inliners_outliers(inliers, outliers, i_label=1, o_label=0, verbose=False):

    temp_outliers = outliers
    temp_inliers  = inliers
    if i_label==1:
        i_labels = np.ones(temp_inliers.shape[0])
    else:
        i_labels = np.zeros(temp_inliers.shape[0])
        
    if o_label==0:
        o_labels = np.zeros(temp_outliers.shape[0])
    else:
        o_labels = np.ones(temp_outliers.shape[0])       
              
    mixed_labels =  np.append(i_labels, o_labels)
    mixed_data = np.vstack((temp_inliers, temp_outliers))

    return mixed_data, mixed_labels

In [28]:
test = np.asarray(sc_id)
ADV_DS_LIST = ("fgsm", "bim-a", "bim-b", "jsma", "cw-l2")
for i, ds_name in enumerate(ADV_DS_LIST):
    adv = id_name+"_"+id_model+"_"+ ds_name
    das = np.load('./advs_new/'+adv+'.npy',allow_pickle=True)
    
    scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        x_test_encoded = encoder.predict(das[i].reshape(1,32,32,3))
        xx = decoder.predict(x_test_encoded)
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood)
    scores, mixed_labels = combine_inliners_outliers(test, ood)
 
    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
    #     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ds_name))
    #     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
    #     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

The results of fgsm:

tnr_at_95_tpr: 76.73107890410937
detection_acc: 85.86553945157968
AUROC:  99.78140797797927


The results of bim-a:

tnr_at_95_tpr: 4.9114331722462445
detection_acc: 49.95571658564812
AUROC:  54.26831266465353


The results of bim-b:

tnr_at_95_tpr: 4.267310789000836
detection_acc: 49.63365539402541
AUROC:  54.32845820640287


The results of jsma:

tnr_at_95_tpr: 5.981136415850228
detection_acc: 50.4905682074501
AUROC:  54.10272957424633


The results of cw-l2:

tnr_at_95_tpr: 64.35472739746544
detection_acc: 79.67736369825771
AUROC:  91.78609481362568




In [23]:
#layer_inx specifies the index of the OODL found by "find_oodl" jupyter file.
REM_TOP_LAYER = -2
NUM_CLASS = 10
if id_name=="MNIST" and id_model=="LeNet":
    layer_inx = 0
    OOD_DS_LIST       = OOD_DS_LIST_MNIST
    #**********************************************************
elif id_name=="CIFAR10":
    OOD_DS_LIST      = OOD_DS_LIST_CIFAR10
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 20
    #**********************************************************
elif id_name=="SVHN":
    OOD_DS_LIST      = OOD_DS_LIST_SVHN
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 24

In [24]:
# Vriance模块(OODL大类别 --> 删除同样的feature maps)
test = np.asarray(sc_id)
for i, ood_ds_name in enumerate(OOD_DS_LIST):
    (_,_),(ood_eva_data,_) = load_dataset(ood_ds_name)
    das = preprocess_images(id_name, ood_eva_data, id_model, verbose=True)
    
    scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        x_test_encoded = encoder.predict(das[i].reshape(1,32,32,3))
        xx = decoder.predict(x_test_encoded)
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood) 
    scores, mixed_labels = combine_inliners_outliers(test, ood) 

    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
#     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ood_ds_name))
#     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
#     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

Preprocessing was done for  CIFAR10 VGG
The results of TINYIMAGENET_RESIZED_32:

tnr_at_95_tpr: 49.8599999995014
detection_acc: 72.4299999992757
AUROC:  83.165592


Preprocessing was done for  CIFAR10 VGG
The results of LSUN_RESIZED:

tnr_at_95_tpr: 65.4399999993456
detection_acc: 80.2199999991978
AUROC:  90.900345


Preprocessing was done for  CIFAR10 VGG
The results of ISUN_PATCHED:

tnr_at_95_tpr: 53.38935574169872
detection_acc: 74.19467787037435
AUROC:  84.80936582633055


Preprocessing was done for  CIFAR10 VGG
The results of SVHN_CROPPED:

tnr_at_95_tpr: 1.1216963736896064
detection_acc: 48.060848186369796
AUROC:  13.46894783343577


Preprocessing was done for  CIFAR10 VGG
The results of G_255:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  100.0


Preprocessing was done for  CIFAR10 VGG
The results of U_255:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  100.0




In [25]:
ood_ds_name = 'foolingimages'

das = np.load('adv_datasets/%s_%s_fooling_images.npy'%(id_name,id_model))

scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])

sc_ood = []
for i in range(len(das)):
        x_test_encoded = encoder.predict(das[i].reshape(1,32,32,3))
        xx = decoder.predict(x_test_encoded)
        score = mark(das[i],xx)
        sc_ood.append(-score)

ood = np.asarray(sc_ood) 
scores, mixed_labels = combine_inliners_outliers(test, ood) 

# print('scores is:', (scores[10000:20000]))
fpr, tpr = nums(scores, mixed_labels)
roc_auc = auc(fpr, tpr)
lens = id_eva_data_processed.shape[0]
FP,TN,TP,FN = ErrorRateAt95Recall1(lens,scores, mixed_labels)
ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
print('The results of %s:\n' %(ood_ds_name))
# print(get_summary_statistics(scores, mixed_labels))
# print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
# print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
print("AUROC: ",ROC*100)
print("\n")

The results of foolingimages:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  99.99299799999999


